# Importing Libraries

In [11]:
from kafka import KafkaProducer
import pyspark
from pyspark import SparkContext, SparkConf
import json
import findspark
findspark.init
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,lower

# Connecting MongoDB and Extracting Data 

In [13]:

database_uri = "mongodb://127.0.0.1:27017/review_data.reviews"
my_spark = SparkSession\
    .builder\
    .appName("MyApp")\
    .config("spark.mongodb.input.uri", database_uri)\
    .config("spark.mongodb.output.uri", database_uri)\
    .config('spark.jars.packages','org.mongodb.spark:mongo-spark-connector_2.12:3.0.1')\
    .getOrCreate()

In [14]:
df = my_spark.read.format('com.mongodb.spark.sql.DefaultSource').load()
df_filtered = df.filter(col("reviewText").like("%alexa%"))
df_filtered.show()                    

+--------------------+----------+-------+--------------------+-----------+--------------+--------------------+--------------+--------+
|                 _id|      asin|overall|          reviewText| reviewTime|    reviewerID|             summary|unixReviewTime|verified|
+--------------------+----------+-------+--------------------+-----------+--------------+--------------------+--------------+--------+
|{644ead17f66a6378...|B017O9P72A|    1.0|The service works...|12 29, 2017| AA4DHYT5YSSIT|       Does not work|    1514505600|   false|
|{644ead17f66a6378...|B017O9P72A|    2.0|I have to tell al...|07 24, 2017|A1ZH6WTMUGWCOU|Commands need to ...|    1500854400|   false|
|{644ead17f66a6378...|B017O9P72A|    3.0|Null message fixe...| 01 5, 2017|A2XD433O61HBZZ|Log out to fix nu...|    1483574400|   false|
|{644ead17f66a6378...|B017O9P72A|    1.0|When I try to lin...|09 15, 2016|A2ZJTS4QMA52IV|Error trying to A...|    1473897600|   false|
|{644ead17f66a6378...|B017O9P72A|    2.0|Horrible,  abs

# Activating the Producer and sending the Data to the consumer

In [84]:
df=df_filtered.toPandas()
producer=KafkaProducer(bootstrap_servers=["localhost:9092"],api_version=(10,0,1))
for i in range(len(df)):
    x=df.iat[i,6]
    res = bytes(x, 'utf-8')
    print(res)
    producer.send("test",res)

b'Does not work'
b'Commands need to get shorter!!!!'
b'Log out to fix null message'
b'Error trying to Authorize'
b'Underwhelming integration and no voice color changing'
b'Alexa turn off bedroom light!'
b'not fully supported'
b'ical works fine you just have to say "alexa ask free busy..."'
b'Amazing, Meow '
b'Surprisingly, the facts are kind of interesting'
b'writing prompt'
b'Works perfectly!'
b'she got it again!'
b'The best app ever'
b'THIS GAME IS AWSOME'
b'=)'
b'Fun, but'
b'Not recoginize words'
b'Needs a limit but love this game'
b'Does not link'
b'Needs less in commands.'
b'Buggy'
b'A Great Skill!!'
b'Has good potential but needs some tweaking!'
b'all in all a fun game to play'
b'Ok fun.'
b'Hard time understand'
b'Fun but needs improvements'
b'Cant understand words'
b'What a great app...'
b'Thank God this skill exists'
b'Great app but.'
b'Not kjv'
b'Almost seems like it gradually loses its skills over time'
b'Yeah'
b'I used to enjoy this app thoroughly until they updated it witho